In [9]:
import os
import czifile
import tifffile
from tqdm import tqdm
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# Path to directory containing the CZI files
czi_dir = 'czi_files'

# Directory to save the TIFF files
# creates subfolder within directory 'TIFF files'
tiff_dir = os.path.join(czi_dir, 'tiff_files')
os.makedirs(tiff_dir, exist_ok=True)

max_width = 10000  # Maximum width after downsampling
max_height = 10000  # Maximum height after downsampling

max_file_size = 1000000000

# Get all files in directory ending with '.czi'
czi_files = [file for file in os.listdir(czi_dir) if file.endswith('.czi')]


# Loop over all files in the directory
for filename in tqdm(czi_files, desc="Converting files"):
    # Construct the full file path
    czi_file = os.path.join(czi_dir, filename)

    # check to see if file size < 1GB
    file_size = os.path.getsize(czi_file)

    # if file is small enough, proceed with conversion and downsampling
    if file_size < max_file_size:

        try:
            # Try to read the CZI file
            with czifile.CziFile(czi_file) as czi:
                img_data = czi.asarray()

        except Exception as e:
            print(f"File {filename} raised an error during processing: {str(e)}. Continuing with next file.")        
            continue
            
        # Construct the output file path (change the extension to .tiff)
        tiff_file = os.path.join(tiff_dir, os.path.splitext(filename)[0] + '.tiff')
        
        # Save as a TIFF file
        tifffile.imsave(tiff_file, img_data)

        try:
        #downsample and save over original tiff file
            with Image.open(tiff_file) as img:
                # Resize the image while preserving aspect ratio
                img.thumbnail((max_width, max_height))
                # Save the resized image
                img.save(tiff_file)
            
            print(f"Successfully downsampled {filename}.")
        
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}. Continuing with next file.")
    
    # if file is too large, skip.
    else:
        print(f"{filename} exceeds max file size. Skipping.")

print("All CZI files have been converted to TIFF and downsampled.")


Converting files:   0%|          | 0/17 [00:00<?, ?it/s]/var/folders/ky/zl0049892b138kmk5t49jgjr0000gp/T/ipykernel_91143/1860912039.py:50: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(tiff_file, img_data)
Converting files:   6%|▌         | 1/17 [00:10<02:54, 10.89s/it]

Successfully downsampled P21-22CPN_R-1a-2023_10_25__0004.czi.


Converting files:  12%|█▏        | 2/17 [00:28<03:43, 14.89s/it]

Successfully downsampled P20_L 1 - 2023_03_10__0016.czi.


Converting files:  18%|█▊        | 3/17 [00:45<03:40, 15.75s/it]

...

Successfully downsampled P20-062 R 1 - 2022_10_31_1671.czi.


Converting files:  94%|█████████▍| 16/17 [04:06<00:16, 16.60s/it]

Successfully downsampled P20-063 R 2 - 2022_10_10_950.czi.


Converting files: 100%|██████████| 17/17 [04:18<00:00, 15.18s/it]

Successfully downsampled P20-063 R 1 - 2022_10_10_949.czi.
All CZI files have been converted to TIFF and downsampled.
